In [1]:
import sqlite3
import pandas as pd
import numpy as np
from plotly import express as px
from sklearn.linear_model import LinearRegression
import calendar

# 1. Create a Database

**Databases** provide us with a structured way to move subsets of data from storage into memory. Our first goal in this blog is to create a database with three tables: **temperatures**, **stations**, and **countries**. Rather than merging introuduced in PIC16A, I am going to keep these as three separate tables in my database. But first of all, let's import these three data sets and take a look at them seperately for the purpose of data cleaning.

In [2]:
temps = pd.read_csv("temps.csv")
temps

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,ACW00011604,1961,-89.0,236.0,472.0,773.0,1128.0,1599.0,1570.0,1481.0,1413.0,1174.0,510.0,-39.0
1,ACW00011604,1962,113.0,85.0,-154.0,635.0,908.0,1381.0,1510.0,1393.0,1163.0,994.0,323.0,-126.0
2,ACW00011604,1963,-713.0,-553.0,-99.0,541.0,1224.0,1627.0,1620.0,1596.0,1332.0,940.0,566.0,-108.0
3,ACW00011604,1964,62.0,-85.0,55.0,738.0,1219.0,1442.0,1506.0,1557.0,1221.0,788.0,546.0,112.0
4,ACW00011604,1965,44.0,-105.0,38.0,590.0,987.0,1500.0,1487.0,1477.0,1377.0,974.0,31.0,-178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359932,ZIXLT622116,1966,2180.0,2040.0,1840.0,1690.0,1430.0,1280.0,1210.0,1460.0,1770.0,1980.0,2090.0,2110.0
1359933,ZIXLT622116,1967,2110.0,1990.0,1890.0,1920.0,1510.0,1350.0,1100.0,1380.0,1660.0,2080.0,1990.0,1910.0
1359934,ZIXLT622116,1968,2180.0,2000.0,1930.0,1820.0,1560.0,1080.0,1370.0,1630.0,1760.0,2180.0,1840.0,2070.0
1359935,ZIXLT622116,1969,2090.0,2150.0,1950.0,1830.0,1410.0,1310.0,1160.0,1460.0,1780.0,2100.0,2040.0,1910.0


In [3]:
stations = pd.read_csv("station-metadata.csv")
stations

,ID,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,57.7667,11.8667,18.0,SAVE
1,AE000041196,25.3330,55.5170,34.0,SHARJAH_INTER_AIRP
2,AEM00041184,25.6170,55.9330,31.0,RAS_AL_KHAIMAH_INTE
3,AEM00041194,25.2550,55.3640,10.4,DUBAI_INTL
4,AEM00041216,24.4300,54.4700,3.0,ABU_DHABI_BATEEN_AIR
...,...,...,...,...,...
27580,ZI000067983,-20.2000,32.6160,1132.0,CHIPINGE
27581,ZI000067991,-22.2170,30.0000,457.0,BEITBRIDGE
27582,ZIXLT371333,-17.8300,31.0200,1471.0,HARARE_BELVEDERE
27583,ZIXLT443557,-18.9800,32.4500,1018.0,GRAND_REEF


In [4]:
countries = pd.read_csv("countries.csv")
countries

,FIPS 10-4,ISO 3166,Name
0,AF,AF,Afghanistan
1,AX,-,Akrotiri
2,AL,AL,Albania
3,AG,DZ,Algeria
4,AQ,AS,American Samoa
...,...,...,...
274,-,-,World
275,YM,YE,Yemen
276,-,-,Zaire
277,ZA,ZM,Zambia


As can be seen in the data sets above, there are few cleaning steps required before incorporating them into our database. To do that, we need the **prepare_df()** function below for the **temps** data set and also column names modification.

In [5]:
def prepare_df(df):
    df = df.set_index(keys=["ID", "Year"])
    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns = {"level_2"  : "Month" , 0 : "Temp"})
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    return(df)

In [6]:
# removing space to favor sql syntax
countries = countries.rename(columns = {"FIPS 10-4":"FIPS","ISO 3166":"ISO","Name":"COUNTRY"})

Now, the **sqlite3** module imported above can help connect to a database. In case the specified database does not exist, instantiating the connection will also create an empty database with the specified name as the following.

In [7]:
conn = sqlite3.connect("blog-1-database.db") # create a database in current directory called blog-1-database.db

When dealing with large data sets like the one named temps above, it's relatively rare that we absolutely have to operate on the entire data set. In most cases, we can work with parts of the data at a time. One good new is that Pandas supplies a nice approach to this using the familiar **pd.read_csv()** function. Supplying a value of **chunksize** will cause **read_csv()** to return not a data frame but an iterator, each of whose elements is a piece of the data with number of rows equal to **chunksize**.

Finally, we're ready to populate a table in our database! Note: The **df.to_sql()** method writes to a specified table in the database (the **conn** object from earlier). We also need to specify **if_exists** to ensure that we add each piece to the table, rather than overwriting them each time.

In [8]:
df_iter = pd.read_csv("temps.csv", chunksize = 100000)
for df in df_iter:
    df = prepare_df(df)
    df.to_sql("temperatures", conn, if_exists = "append", index = False)

Let's similarly add (**left join**) two more table for the remaining metadata in our database. Noticing that **stations** and **countries** are pretty small data sets, we don't need to worry about reading it in by chunks.

In [9]:
stations.to_sql("stations", conn, if_exists = "replace", index = False)

In [10]:
countries.to_sql("countries", conn, if_exists = "replace", index = False)

Now we have a database containing three tables. Let's just check that this is indeed the case through executeing SQL commands by the cursor interacted with the database.

In [11]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('temperatures',), ('stations',), ('countries',)]


To get more detailed information about the items in each table, the following execution is a good way to check that we actually populated our database correctly.

In [12]:
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "temperatures" (
"ID" TEXT,
  "Year" INTEGER,
  "Month" INTEGER,
  "Temp" REAL
)
CREATE TABLE "stations" (
"ID" TEXT,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "STNELEV" REAL,
  "NAME" TEXT
)
CREATE TABLE "countries" (
"FIPS" TEXT,
  "ISO" TEXT,
  "COUNTRY" TEXT
)


To close the connection, we run the following command.

In [13]:
conn.close()

# 2. Write a Query Function

The next goal here is to write a function called **query_climate_database()** which accepts four arguments:

- `country`
- `year_begin` and `year_end`
- `month`

The return of the function is a Pandas dataframe of temperature readings for the specified country, in the specified date range, in the specified month of the year. This dataframe should also include columns for:

- The station name.
- The latitude of the station.
- The longitude of the station.
- The name of the country in which the station is located.
- The year in which the reading was taken.
- The month in which the reading was taken.
- The average temperature at the specified station during the specified year and month. (Note: the temperatures in the raw data are already averages by month)

In [14]:
def query_climate_database(country, year_begin, year_end, month):
    """
    This function will access the database and use the four arguments to help take a specified subset and return it as a DataFrame.
    Once it's done, close the connection.
    """
    # connect to the database
    conn = sqlite3.connect("blog-1-database.db")

    cmd = \
    """
    SELECT S.NAME, S.LATITUDE, S.LONGITUDE, C.Country, T.Year, T.Month, T.Temp  
    FROM stations S
    LEFT JOIN temperatures T ON S.ID = T.ID
    LEFT JOIN countries C ON SUBSTRING (T.id, 1, 2) = C.FIPS
    WHERE T.year >= {year_begin} AND T.year <= {year_end} AND T.Month = {month} AND C.COUNTRY = '{country}'
    """.format(year_begin=year_begin, year_end=year_end, month=month, country=country)
    
    # return a DataFrame corresponding to the result set of the query string
    df = pd.read_sql_query(cmd, conn)
    # close the connection
    return df

Here's a demonstration:

In [15]:
query_climate_database(country = "India", 
                       year_begin = 1980, 
                       year_end = 2020,
                       month = 1)

,NAME,LATITUDE,LONGITUDE,COUNTRY,Year,Month,Temp
0,PBO_ANANTAPUR,14.583,77.633,India,1980,1,23.48
1,PBO_ANANTAPUR,14.583,77.633,India,1981,1,24.57
2,PBO_ANANTAPUR,14.583,77.633,India,1982,1,24.19
3,PBO_ANANTAPUR,14.583,77.633,India,1983,1,23.51
4,PBO_ANANTAPUR,14.583,77.633,India,1984,1,24.81
...,...,...,...,...,...,...,...
3147,DARJEELING,27.050,88.270,India,1983,1,5.10
3148,DARJEELING,27.050,88.270,India,1986,1,6.90
3149,DARJEELING,27.050,88.270,India,1994,1,8.10
3150,DARJEELING,27.050,88.270,India,1995,1,5.60


# 3. Write a Geographic Scatter Function for Yearly Temperature Increases

In this part, suppose we want to know *How does the average yearly change in temperature vary within a given country?*

To answer this question, we write a function called **temperature_coefficient_plot()**. This function should accept five explicit arguments, and an undetermined number of keyword arguments.

- `country`, `year_begin`, `year_end`, and `month`
- `min_obs`, the minimum required number of years of data for any given station
- `**kwargs`, , additional keyword arguments passed to **px.scatter_mapbox()**. These can be used to control the colormap used, the mapbox style, etc.

The output of this function should be an interactive geographic scatterplot, constructed using Plotly Express, with a point for each station, such that the color of the point reflects an estimate of the yearly change in temperature during the specified month and time period at that station. A reasonable way to do this is to compute the first coefficient of a linear regression model at that station. 

In [16]:
def coef(data_group):
    x = data_group[["Year"]] # 2 brackets because X should be a df
    y = data_group["Temp"]   # 1 bracket because y should be a series
    LR = LinearRegression()
    LR.fit(x, y)
    return round(LR.coef_[0], 4)

In [31]:
def temperature_coefficient_plot(country, year_begin, year_end, month, min_obs, **kwargs):
    """
    This function will apply the inputted arguments to help take a specified subset as df.
    After filtering out data for any given station that are less than the minimum required number of years, we calculate
    the linear corrleation coefficient between the temperature and the year. With these results,
    it will return an interactive geographic scatterplot using Plotly Express.
    """
    df = query_climate_database(country, year_begin, year_end, month)
    
    df['obs'] = 1
    # Count the size of the grouped station
    df['obs'] = df.groupby(['NAME'])['obs'].transform(np.sum)
    # fitler out the data that does not satisfy the minimum requirement.
    df = df[df['obs'] >= min_obs]
    
    # regroup the data and apply coefficient function
    df = df.groupby(['NAME', 'LATITUDE', 'LONGITUDE', 'obs']).apply(coef)
    df = df.reset_index()
    df = df.rename(columns = {0: "Estimated Yearly Increase (°C)"})
    
    # plot the mapbox
    fig = px.scatter_mapbox(df, 
                        lat = "LATITUDE",
                        lon = "LONGITUDE", 
                        hover_name = "NAME", # display the station name in bold as the tooltip title
                        color = "Estimated Yearly Increase (°C)", # color by the average yearly change in temperature
                        color_continuous_midpoint = 0, # center the colorbar as 0
                        title = "Estimates of yearly increase in temperature in {month} <br> for stations in {country}, years {y_b} - {y_e}".format(
                        month = calendar.month_name[1], country = country, y_b = year_begin, y_e = year_end),
                        **kwargs)
    # line break in the title of the colorbar
    fig.layout.coloraxis.colorbar.title = 'Estimated Yearly <br> Increase (°C)'
    return fig

For example, after writing the function above, it is able to create a plot of estimated yearly increases in temperature during the month of January, in the interval 1980-2020, in India, as follows:

In [32]:
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig = temperature_coefficient_plot("India", 1980, 2020, 1, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)
fig.show()

# 4. Create Two More Interesting Figures

Question 4.1:
`How does the average yearly change in temperature vary by the seasonal effect within a given country and a given list of stations?`

In this part, we'll need to modify the **query_climate_database()** function above a bit and also create a function **find_season()** to help us determine a season (spring, summer, autumn, winter) by a given month. Since the season is varied between the Northern and Southern Hemispheres. To make things simple, let's just assume all seasons are following the Northern Hemisphere, such that winter is from December to February, and so forth.

In [19]:
def query_climate_database2(country, year_begin, year_end, Station_list):
    # connect to the database
    conn = sqlite3.connect("blog-1-database.db")

    cmd = \
    """
    SELECT S.NAME, C.Country, T.Year, T.Month, T.Temp  
    FROM stations S
    LEFT JOIN temperatures T ON S.ID = T.ID
    LEFT JOIN countries C ON SUBSTRING (T.id, 1, 2) = C.FIPS
    WHERE T.year >= {year_begin} AND T.year <= {year_end} AND C.COUNTRY = '{country}' AND S.NAME IN {S_l}
    """.format(year_begin=year_begin, year_end=year_end, country=country, S_l = Station_list)
    
    # return a DataFrame corresponding to the result set of the query string
    df = pd.read_sql_query(cmd, conn)
    # close the connection
    conn.close()
    return df

In [20]:
def find_season(m) : 
    # Taken all the possible
    # month numbers in the list.
    seasons = [[12 , 1 , 2], [3 , 4 , 5], [6 , 7 , 8], [9 , 10 , 11]]          
    # Matching the month number
    if m in seasons[0]:
        return "WINTER"
    elif m in seasons[1]:
        return "SPRING"
    elif m in seasons[2]:
        return "SUMMER"
    else:
        return "AUTUMN"

In [34]:
def seaonal_temperature_scatter(country, year_begin, year_end, Station_list):
    
    df = query_climate_database2(country, year_begin, year_end, Station_list)
    
    df['Season'] = df['Month'].apply(find_season)
    
    fig = px.scatter(df, x = "Year", y = "Temp", color = "NAME", facet_col = "Season",
                     width = 900, height = 450, opacity = 0.5,
                     title = "Temperature by Season for stations {S_l} <br> in {country}, years {y_b} - {y_e}".format(
                         S_l = Station_list, country = country, y_b = year_begin, y_e = year_end))
    return fig

In [35]:
fig = seaonal_temperature_scatter("Japan", 1980, 2020, ('TOKYO','KYOTO', 'HIROSHIMA', 'KOBE', 'OSAKA'))
fig.show()

Question 4.2:
`How does the temperature distribution in summary statstics by season vary within a given country and a list of stations?`

In this part, we can continue to apply the **query_climate_database2()** function to help us achieve this task. To display the summary distribution, we use **px.box()** to draw the boxplot.

In [36]:
def seaonal_temperature_boxplot(country, year_begin, year_end, Station_list):
    
    df = query_climate_database2(country, year_begin, year_end, Station_list)
    
    df['Season'] = df['Month'].apply(find_season)
    
    fig = px.box(df, x = "Season", y = "Temp", color = "NAME",
                 title = "Temperature Summary Distribution by Season <br> for stations {S_l} in {country}, years {y_b} - {y_e}".format(
                        S_l = Station_list, country = country, y_b = year_begin, y_e = year_end))

    return fig

In [37]:
fig = seaonal_temperature_boxplot("China", 1980, 2020, ('BEIJING', 'SHANGHAI', 'GUANGZHOU', 'SHENZHEN'))
fig.show()